<a href="https://colab.research.google.com/github/xiaoyi-yuxiao/python-code-for-FinancialAnalysis-book/blob/master/2020_6_8_chap9_dowjones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests_html

In [ ]:
pip install yahoo_fin

In [ ]:
pip install bs4

In [ ]:
pip install requests

In [ ]:
from yahoo_fin.stock_info import *
import pandas as pd
from tabulate import tabulate
import datetime as dt#get current date
from google.colab import files
import requests
from bs4 import BeautifulSoup
import numpy as np

In [ ]:
def html_beautifulsoup(url):
    response = requests.get(url)
    html = response.content

    #beautifulsoup html will not look like soure
    #review beautifulsoup html to use find and findall
    soup = BeautifulSoup(html,"lxml")
    return soup

#soup_html: soup html is not same as source html
def yahoo_description(soup_html):
    element = soup_html.find('h1',class_='D(ib) Fz(16px) Lh(18px)')
    return element.text  

In [ ]:
previousday = dt.date.today()-dt.timedelta(days=1)# get the date of previous day
Default_end = previousday.strftime("%Y-%m-%d") # change the format of the default datetime 
years_ago = dt.date.today()-dt.timedelta(days=5 * 365)
Default_start = years_ago.strftime("%Y-%m-%d")

# request user input
print("Calculate the 4 statistical moments of the rate of returns of the Dow 30 stocks.")
Start = input('Enter start date(' + 'Default = 5 Years ago, ' + Default_start + ')-->') or Default_start
T_interval = input('Enter date interval(1d,1wk,1mo Default = 1mo)-->') or '1mo'

Calculate the 4 statistical moments of the rate of returns of the Dow 30 stocks.
Enter start date(Default = 5 Years ago, 2015-06-25)-->2015-06-25
Enter date interval(1d,1wk,1mo Default = 1mo)-->1mo


Display price

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
tickers = tickers_dow() # procedure from yahoo_fin
Data = []
Url = []

#retrieving stock description from Yahoo!Finance
i = 1
for ticker in range(len(tickers)):
  Data.append(get_data(tickers[ticker], start_date = Start,index_as_date = True, interval = T_interval)['adjclose'])
  soup_html = html_beautifulsoup("https://finance.yahoo.com/quote/"  + tickers[ticker])
  stock_name = yahoo_description(soup_html)
  Url.append('https://finance.yahoo.com/quote/'+tickers[ticker])
  tickers[ticker] = stock_name
  print(str(i) + ' of 30   Getting ' + stock_name + ' data ...')
  i=i+1

Data = pd.concat(Data,axis=1)
Data.columns = tickers

1 of 30   Getting AAPL - Apple Inc. data ...
2 of 30   Getting AXP - American Express Company data ...
3 of 30   Getting BA - The Boeing Company data ...
4 of 30   Getting CAT - Caterpillar Inc. data ...
5 of 30   Getting CSCO - Cisco Systems, Inc. data ...
6 of 30   Getting CVX - Chevron Corporation data ...
7 of 30   Getting DIS - The Walt Disney Company data ...
8 of 30   Getting DOW - Dow Inc. data ...
9 of 30   Getting GS - The Goldman Sachs Group, Inc. data ...
10 of 30   Getting HD - The Home Depot, Inc. data ...
11 of 30   Getting IBM - International Business Machines Corporation data ...
12 of 30   Getting INTC - Intel Corporation data ...
13 of 30   Getting JNJ - Johnson & Johnson data ...
14 of 30   Getting JPM - JPMorgan Chase & Co. data ...
15 of 30   Getting KO - The Coca-Cola Company data ...
16 of 30   Getting MCD - McDonald's Corporation data ...
17 of 30   Getting MMM - 3M Company data ...
18 of 30   Getting MRK - Merck & Co., Inc. data ...
19 of 30   Getting MSFT - M

In [ ]:
#print DOW 30  First 4 Statistical Moments
Ret = Data.pct_change()
summury = pd.concat([Ret.mean(),Ret.std(),Ret.skew(),Ret.kurtosis(),np.divide(Ret.std(),Ret.mean())],axis=1)
summury.columns = ['Mean','standard deviation','skewness','kurtosis','coefficient of variation']
summury.insert(0, "Ticker&stock description", tickers, True) 
summury.insert(6, "Url", Url, True) 

print("")
print("                                                                                First Four Moments of Rates of Returns")
print("                                                                                    Dow Jones Industrial 30 Firms")
print ("                                                                                       " + Start + ' to ' + Default_end            )
print("")
print(tabulate(summury,headers=summury.columns,tablefmt="github",showindex=False))
print("")
print("Dow 30 List Source: https://finance.yahoo.com/quote/^DJI/components")


                                                                                First Four Moments of Rates of Returns
                                                                                    Dow Jones Industrial 30 Firms
                                                                                       2015-06-25 to 2020-06-22

| Ticker&stock description                          |         Mean |   standard deviation |   skewness |   kurtosis |   coefficient of variation | Url                                  |
|---------------------------------------------------|--------------|----------------------|------------|------------|----------------------------|--------------------------------------|
| AAPL - Apple Inc.                                 |  0.0231772   |            0.080734  | -0.36072   | -0.214222  |                    3.48333 | https://finance.yahoo.com/quote/AAPL |
| AXP - American Express Company                    |  0.00814628  |            0.0665038 | -1.

In [ ]:
#Save dow 30 report to download folder
summury.to_csv('Dow30FourMoments.csv')
print("Saving Dow 30 report to 'Download' folder") 
files.download('Dow30FourMoments.csv')
print("Finish saving Dow 30 report to 'Download' folder")

Saving Dow 30 report to 'Download' folder


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Finish saving Dow 30 report to 'Download' folder
